# Core concepts

## Goal of this notebook

- Learn about the most important data structures in $\text{FeO}_\text{s}$ using the Peng-Robinson equation of state.

## The equation of state object

Before we can compute a thermodynamic property, we have to decide what equation of state to use.
In $\text{FeO}_\text{s}$, an **equation of state** is an object that contains information about the system (which substances), the parameters needed, and the algorithms to compute the Helmholtz energy.

Let's define a simple Peng-Robinson equation of state. We can import the `PengRobinson` object from `feos_core.cubic`.

In [47]:
from feos_core.cubic import PengRobinson

PengRobinson?

Init signature: PengRobinson(self, /, *args, **kwargs)
Docstring:     
A simple version of the Peng-Robinson equation of state.

Parameters
----------
parameters : PengRobinsonParameters
    The parameters of the Peng-Robinson equation of state to use.

Returns
-------
PengRobinson
Type:           type
Subclasses:     


The `PengRobinson` equation of state constructor takes a single argument as input, the parameters which have the data type `PengRobinsonParameters`. Those are imported also from the `feos_core.cubic` module.

The way an equation of state handles parameters is in control of the implementor of the equation of state and can -- in general -- be different for each equation of state.

$\text{FeO}_\text{s}$ offers some convenient tools, e.g. to read parameters from a json file. Without going into more detail, we will use this functionality in this example. 

Let's start with a single component.

In [48]:
from feos_core.cubic import PengRobinsonParameters

parameters = PengRobinsonParameters.from_json(['propane'], 'peng-robinson.json')
parameters

PureRecord(
	identifier=Identifier(cas=74-98-6, name=propane, iupac_name=propane, smiles=CCC, inchi=InChI=1/C3H8/c1-3-2/h3H2,1-2H3, formula=C3H8),
	molarweight=44.0962,
	model_record=PengRobinsonRecord(tc=369.96 K, pc=4250000 Pa, acentric factor=0.153,
)

k_ij:
[[0]]

Now that we have the parameters set up, we can instantiate our equation of state object.

In [49]:
eos = PengRobinson(parameters)

## A thermodynamic state

We now have an equation of state that contains the substance(s) we are interested in as well as the algorithms to compute properties. Next, we have to specify the thermodynamic conditions at which we want to compute our properties.

To that, we have to build a `State` object which located in the same module, `feos_core.cubic`.
There are a lot of ways to define a thermodynamic state.
You can use the documentation (or the docstring) to find out about all possible ways.

Here, let's build a state at given temperature and pressure.
To do that, we need to import yet another module: the `feos_core.si` module contains data types that capture dimensioned quantities. Most interfaces in $\text{FeO}_\text{s}$ use these data types.

Creating a state is always the same: the first argument is the equation of state followed by the control variables. If no amount of substance is specified, it is set to the inverse of Avogradro's number.

In [50]:
from feos_core.cubic import State
from feos_core.si import * # SI units and constants 

state_tp = State(eos, temperature=300*KELVIN, pressure=1*BAR)
state_tp

|temperature|density|
|-|-|
|300.00000 K|40.75540  mol/m³|

## Thermodynamic properties
The main work is done. A `State` object has a method for each of its thermodynamic properties. Let's compute some.

In [51]:
state_tp.pressure()

99.99999999999994 kPa

In [52]:
state_tp.molar_enthalpy()

19.050830649458582 kJ/mol

In [53]:
state_tp.speed_of_sound()

187.58774865257948  m/s

In [54]:
state_tp.joule_thomson()

-7.101416421069843e-5 m kg^-1 s^2 K

In [55]:
state_tp.isentropic_compressibility()

1.5812651174117774e-5 m kg^-1 s^2

In [56]:
state_tp.isothermal_compressibility()

1.0167625928735927e-5 m kg^-1 s^2

## `State` at critical conditions

There are other ways to create a `State` object. For example, we can compute the critical point.

In [63]:
state_cp = State.critical_point(eos)

print('critical temperature: {:2f} K'.format(
    state_cp.temperature / KELVIN
))
print('critical pressure   : {:2f} MPa'.format(
    state_cp.pressure() / (MEGA * PASCAL)
))
print('critical density    : {:2f} g/cm3'.format(
    state_cp.mass_density() / (GRAM / (CENTI * METER)**3)
))

critical temperature: 369.950617 K
critical pressure   : 4.249678 MPa
critical density    : 0.198186 g/cm3


## Phase equilibirum

Another important property is phase equilibrium which generates two or more states that are in equilibrium.
For pure substance systems, we can either provide the temperature or the pressure:

In [68]:
from feos_core.cubic import PhaseEquilibrium

PhaseEquilibrium.pure(eos, 0.9 * state_cp.pressure())

||temperature|density|
|-|-|-|
|phase 1|363.92630 K|2.84888 kmol/m³|
|phase 2|363.92630 K|6.45189 kmol/m³|


In [69]:
PhaseEquilibrium.pure(eos, 0.9 * state_cp.temperature)

||temperature|density|
|-|-|-|
|phase 1|332.95556 K|1.13281 kmol/m³|
|phase 2|332.95556 K|9.60459 kmol/m³|


The `PhaseEquilibrium` object contains the `State` objects which we can access via `liquid` or `vapor` and which we can use to compute any property, e.g. the enthalpy of vaporization.

In [71]:
vle = PhaseEquilibrium.pure(eos, 0.9 * state_cp.temperature)
h_vap = vle.vapor.molar_enthalpy() - vle.liquid.molar_enthalpy()
h_vap

11.361380517650792 kJ/mol

## Concluding remkars

Hopefully you found this example helpful. If you have comments, critique or feedback, please let us know and consider opening an [issue on github](https://github.com/feos-org/feos/issues).